In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sc
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
airplanes_df = pd.read_csv('../../data/Clean_Dataset.csv') # read our CSV file
airplanes1_df = airplanes_df#.drop(['Unnamed: 0'], axis=1) #drops the first  column (id)
airplanes_train, airplanes_test = train_test_split(airplanes1_df, test_size=0.2, random_state=4) # on divise les données en 2 sets
###
#airplane_train and _test sont de type <class 'pandas.core.frame.DataFrame'> (the last column is price)

In [222]:
class Node():
    
    def __init__(self, X):
        self.X_parent = X
        self.X_left = None
        self.X_right = None
        self.depth = 0
        self.column = None
        self.condition = None
        self.MSE = mean_squared_error([np.mean(self.X_parent['price'])]*len(self.X_parent['price']) , self.X_parent['price'])
        print(self.MSE)
    
    def separate_from_class(self,column, condition): # pour separateurs par classification
        X_gauche = self.X_parent[self.X_parent[column] == condition]
        X_droite = self.X_parent[self.X_parent[column] != condition]
        
        size_G = len(X_gauche['price'])
        size_D = len(X_droite['price'])
        MSE_G = mean_squared_error([np.mean(X_gauche['price'])]*size_G , X_gauche['price'])
        MSE_D = mean_squared_error([np.mean(X_droite['price'])]*size_D , X_droite['price'])
        return ( MSE_G*size_G + MSE_D*size_D )/(size_G + size_D)
        
    def separate_from_int(self,column,condition): # pour separateurs par rapport a une valeur
        X_gauche = self.X_parent[self.X_parent[column] < float(condition)]
        X_droite = self.X_parent[self.X_parent[column] >= float(condition)]
        
        size_G = len(X_gauche['price'])
        size_D = len(X_droite['price'])
        MSE_G = mean_squared_error([np.mean(X_gauche['price'])]*size_G , X_gauche['price'])
        MSE_D = mean_squared_error([np.mean(X_droite['price'])]*size_D , X_droite['price'])
        return ( MSE_G*size_G + MSE_D*size_D )/(size_G + size_D)
    
    def best_split(self):
        best_MSE = noeud.MSE
        best_cond = noeud.condition
        best_column = noeud.column
        for column in self.X_parent.select_dtypes(include=np.number).columns.drop(["Unnamed: 0","price"]).tolist():
            #print(column)
            for trait in np.sort(airplanes_df[column].unique())[1:]:
                MSE = self.separate_from_int(column,trait)
                if ( MSE < best_MSE):
                    best_MSE = MSE
                    best_cond = trait
                    best_column = column
                    # print("MSE: ", MSE,"\t cond:", column,"<", trait )
        for column in self.X_parent.select_dtypes(include=['object']).columns.drop(["flight"]).tolist() :
            #print(column)
            for trait in np.sort(airplanes_df[column].unique()):
                MSE = self.separate_from_class(column,trait)
                if ( MSE < best_MSE):
                    best_MSE = MSE
                    best_cond = trait
                    best_column = column
                    # print("MSE: ", MSE,"\t cond:", column,"==", trait )
        self.MSE = best_MSE
        self.condition = best_cond
        self.column = best_column
            

In [ ]:
class Tree(X):
    def __init__(self,X):
        
    def create_tree

In [220]:
noeud = Node(airplanes_df)
noeud.best_split()
print (noeud.condition,noeud.column)

515186926.9843313
duration
days_left
airline
source_city
departure_time
stops
arrival_time
destination_city
class
Business class


515186926.9843313

 airline
SpiceJet 	 508489376.3010697
AirAsia 	 499194475.898759

 flight
SG-8709 	 515071308.56803
SG-8157 	 515077684.1263032

 source_city
Delhi 	 514221832.8445954
Mumbai 	 515097074.8344138

 departure_time
Evening 	 515154398.4938372
Early_Morning 	 515109838.9350497

 stops
zero 	 497117962.6943945
one 	 494597450.6020168

 arrival_time
Night 	 514973699.4261957
Morning 	 514711457.17155665

 destination_city
Mumbai 	 515129765.1653341
Bangalore 	 515085229.36009145

 class
Economy 	 62037785.990201846
Business 	 62037785.990201846

 duration
2.17 	 513704178.92390436
2.33 	 514473615.45160544

 days_left
1 	 515183740.83549243
2 	 514005572.1863627

 price
5953 	 514919762.6177916
5956 	 515149770.70521724


In [126]:

noeud.X_parent.select_dtypes(include=np.number).columns.drop(["Unnamed: 0","price"]).tolist()

['duration', 'days_left']

In [127]:

noeud.X_parent.select_dtypes(include=['object']).columns.tolist()

['airline',
 'flight',
 'source_city',
 'departure_time',
 'stops',
 'arrival_time',
 'destination_city',
 'class']